# Information regarding the data that this notebook refers to

In [1]:
# fill out the following fields. Check that input data and notebook name match.

# reference vehicle_fleet_data.xlsx notebook metadata to select notebook_id
notebook_id= '003'
name_string= '_s_GL_2005_2015_cl(cv)_oica'

nb_name= notebook_id + name_string +'.ipynb'
nb_input_workbook= 'in' + name_string + '.xlsx'
nb_output_workbook= notebook_id + '.xlsx'
nb_stock_or_flow= 'stock'
nb_geography= '001'
nb_start_time= '2005'
nb_stop_time= '2015'
nb_attribute_1= 'commercial vehicles'
nb_attribute_2= 'registered'
nb_attribute_3= ''
nb_attribute_4= ''
nb_data_source= 'OICA'
nb_data_source_url= 'http://www.oica.net/category/vehicles-in-use/'
nb_comment= 'original data believed to be in units of thousands'


In [2]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib as plt

In [3]:
# write the notbook metadata into a data frame
notebook_metadata = pd.DataFrame({'notebook_name':nb_name,
                                 'input_file': nb_input_workbook, 
                                 'output_file': nb_output_workbook, 
                                 'source': nb_data_source, 
                                 'geography': nb_geography,
                                 'start_time': nb_start_time,
                                 'stop_time': nb_stop_time, 
                                 'attribute1': nb_attribute_1, 
                                 'attribute2': nb_attribute_2,
                                 'attribute3': nb_attribute_3,
                                 'attribute4': nb_attribute_4,
                                 'source_url' : nb_data_source_url,
                                 'comment': nb_comment}, index=[notebook_id])

## Reading in the excel data and merging the sheets into one dataframe with category info attached to row data

In [4]:
# read in data,
xls = pd.ExcelFile(nb_input_workbook)

In [5]:
# read in data, set the header to be the desired column titles
df = pd.read_excel(xls, header = 5)
df.head(2) # view the data frame

,REGIONS/COUNTRIES,Unnamed: 1,Unnamed: 2,Unnamed: 3,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,Unnamed: 15,Variation 2015/2014
0,NaN,NaN,NaN,NaN,in thousand units,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EUROPE,NaN,NaN,NaN,45053.2,46598.161,47999.341,49363.389,49562.42,50241.362,51097.143,52228.084,52649.729,53293.14,53966.294,NaN,0.012631


In [6]:
# use .dropna() to drop the rows and columns with no data. 
# thresh=2 drops columns that do contain up to 2 pieces of non NAN values
df.dropna(axis=1, thresh=2, inplace=True)
df.dropna(axis=0, thresh=2, inplace=True)
#drop columns or rows that will not be useful
df.drop(columns= 'Variation 2015/2014', inplace= True)

## Import dictionary for country codes and replace country names with 3 digit codes

In [7]:
# rename region to 'geo'
df.rename(columns={'REGIONS/COUNTRIES':'geo'},inplace= True)
df.reset_index(drop=True, inplace=True) # reset the idex
df.head(2)

,geo,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,EUROPE,45053.2,46598.161,47999.341,49363.389,49562.420,50241.362,51097.143,52228.084,52649.729,53293.140,53966.294
1,EU 28 countries + EFTA,34748.8,35793.506,36583.103,37375.604,37452.621,37782.755,38258.398,38264.743,38384.255,38653.294,39182.700


In [8]:
xls = pd.ExcelFile('metadata_vehicle_fleet.xlsx')
geographyMetadata = xls.parse('geography_metadata', skiprows=1, index_col=None)
geographyMetadata.drop('id', axis=1, inplace=True)

missing = []
edited = []

for i in range(1, len(df['geo'])):
    for j in range(0, len(geographyMetadata['name'])): # we need to use different indexes here because the dataframes might have different lengths
        if df['geo'][i].lower() not in geographyMetadata['name'].str.lower().tolist(): # if the country is not in name we try checking in the other cols
            missing.append(df['geo'][i])

            if df['geo'][i].lower() == geographyMetadata['alternate name1'][j].lower():
                df['geo'][i] = geographyMetadata['name'][j]
                edited.append(geographyMetadata['name'][j])
            elif df['geo'][i].lower() == geographyMetadata['alternate name2'][j].lower():
                df['geo'][i] = geographyMetadata['name'][j]
                edited.append(geographyMetadata['name'][j])
            elif df['geo'][i].lower() == geographyMetadata['alternate name3'][j].lower():
                df['geo'][i] = geographyMetadata['name'][j]
                edited.append(geographyMetadata['name'][j])
            elif df['geo'][i].lower() == geographyMetadata['alternate name4'][j].lower():
                df['geo'][i] = geographyMetadata['name'][j]
                edited.append(geographyMetadata['name'][j]) 

print('missing:'+ str(set(missing)))    # This is the list of countries that are not in the names column of the metadata sheet
print('edited:' + str(set(edited)))    # This is the list of countries that were edited in the loop


<ipython-input-8-14d5a5488c59>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['geo'][i] = geographyMetadata['name'][j]
<ipython-input-8-14d5a5488c59>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['geo'][i] = geographyMetadata['name'][j]


missing:{'BRUNEI', 'RUSSIA, TURKEY & OTHER EUROPE', 'CENTRAL & SOUTH AMERICA', 'MACEDONIA', 'EU 28 countries + EFTA', 'MOLDAVIA', 'HONG-KONG', 'ALL COUNTRIES', 'BOSNIA', 'EUROPE NEW MEMBERS', 'AMERICA', 'AFRICA', 'RUSSIA', 'CONGO KINSHASA ', 'CZECH REPUBLIC', 'NAFTA', 'AZERBAIDJAN', 'EU 15 countries + EFTA', 'PALESTINE', 'ASIA/OCEANIA/MIDDLE EAST'}
edited:{'State of Palestine', 'United States of America', 'Bosnia and Herzegovina', 'Russian Federation', 'Czechia', 'Azerbaijan', 'Hong Kong', 'Moldova', 'Brunei Darussalam', 'North Macedonia'}


In [9]:
#find contries which did not match 
df.loc[~df['geo'].str.lower().isin(geographyMetadata.name.str.lower())]

,geo,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,EUROPE,45053.2,46598.161000,47999.341000,49363.389000,49562.420000,50241.362,51097.143000,52228.084000,52649.729,53293.140,53966.294
1,EU 28 countries + EFTA,34748.8,35793.506000,36583.103000,37375.604000,37452.621000,37782.755,38258.398000,38264.743000,38384.255,38653.294,39182.700
2,EU 15 countries + EFTA,29441.4,30359.283000,30788.574000,31131.387000,31086.289000,31282.767,31544.028000,31470.890000,31475.788,31710.740,31984.333
21,EUROPE NEW MEMBERS,5307.44,5434.223000,5794.529000,6244.217000,6366.332000,6499.988,6714.370000,6793.853000,6908.467,6942.554,7198.367
35,"RUSSIA, TURKEY & OTHER EUROPE",10304.4,10804.655000,11416.238000,11987.785000,12109.799000,12458.607,12838.745000,13963.341000,14265.474,14639.846,14783.594
47,NAFTA,112824,118210.000000,122523.000000,123625.000000,126560.000000,129471.000,131954.000000,141371.459000,143623.173,148373.844,153435.383
51,CENTRAL & SOUTH AMERICA,12359.1,12977.394000,13849.887000,14934.235000,15581.608000,16714.118,17976.278000,19128.881000,20033.167,21195.292,22029.622
79,ASIA/OCEANIA/MIDDLE EAST,59429.9,60709.613875,64465.811725,66367.137667,69086.885333,72183.531,75716.050667,81663.431333,86759.792,91673.850,91765.805
118,AFRICA,8507.93,8992.885000,9700.105000,10680.162000,11165.967000,11517.160,11874.892000,12328.691000,12778.751,13288.787,13992.905
126,CONGO KINSHASA,689.2,699.500000,704.000000,707.900000,709.000000,711.000,726.000000,741.000000,760.000,780.000,800.000


In [10]:
# Drop codes that are not in code list (after checking for completeness)
df = df.loc[df['geo'].str.lower().isin(geographyMetadata.name.str.lower())]

In [11]:
df.geo.unique()

array(['AUSTRIA', 'BELGIUM', 'DENMARK', 'FINLAND', 'FRANCE', 'GERMANY',
       'GREECE', 'ICELAND', 'IRELAND', 'ITALY', 'LUXEMBOURG',
       'NETHERLANDS', 'NORWAY', 'PORTUGAL', 'SPAIN', 'SWEDEN',
       'SWITZERLAND', 'UNITED KINGDOM', 'BULGARIA', 'CROATIA', 'CYPRUS',
       'Czechia', 'ESTONIA', 'HUNGARY', 'LATVIA', 'LITHUANIA', 'MALTA',
       'POLAND', 'ROMANIA', 'SLOVAKIA', 'SLOVENIA', 'ALBANIA', 'BELARUS',
       'Bosnia and Herzegovina', 'GEORGIA', 'North Macedonia', 'Moldova',
       'Russian Federation', 'SERBIA', 'TURKEY', 'UKRAINE',
       'United States of America', 'CANADA', 'MEXICO',
       'UNITED STATES OF AMERICA', 'ARGENTINA', 'BAHAMAS', 'BARBADOS',
       'BELIZE', 'BERMUDA', 'BOLIVIA', 'BRAZIL', 'CHILE', 'COLOMBIA',
       'COSTA RICA', 'CUBA', 'DOMINICAN REPUBLIC', 'ECUADOR',
       'EL SALVADOR', 'GUATEMALA', 'HAITI', 'HONDURAS', 'JAMAICA',
       'NICARAGUA', 'PANAMA', 'PARAGUAY', 'PERU', 'PUERTO RICO',
       'SURINAME', 'TRINIDAD AND TOBAGO', 'URUGUAY', 'VENEZU

## rearrange data into proper format

In [12]:
#.melt() pivots the table bringing the column headers into a new attribute
melted = df.melt(id_vars=['geo'], var_name= 'year')
melted

,geo,year,value
0,AUSTRIA,2005,367
1,BELGIUM,2005,674.465
2,DENMARK,2005,479
3,FINLAND,2005,86.69
4,FRANCE,2005,6198
...,...,...,...
1546,TOGO,2015,58
1547,TUNISIA,2015,460
1548,UGANDA,2015,340
1549,ZAMBIA,2015,120


In [13]:
# data orginially in thousands, convert to nr
melted.value *= 1000

## structuring the data into format of datastructure


In [14]:
#rename the data frame to df
df=melted
# add in a column of indexes
df.index = notebook_id + df.index.astype(str).str.zfill(9)

In [15]:
#rename or add all necessary columns
df.loc[:,'year_of_measurement']= df.loc[:,'year']
# date of measurement is concatonated to year of measurement to achieve desired format
df.loc[:,'date_of_measurement']='' #df['year_of_measurement'].astype(str) + '-03-31'
df.head(2)

,geo,year,value,year_of_measurement,date_of_measurement
003000000000,AUSTRIA,2005,367000,2005,
003000000001,BELGIUM,2005,674465,2005,


In [16]:
df.loc[:,'geo']= df.geo.str.title()
df.loc[:,'process']= 'r'
df.loc[:,'vehicle_class']= 'OICV'
df.loc[:,'vehicle_segment']='all'
df.loc[:,'motor_energy']= 'all'
df.loc[:,'model_year']= 'all'
df.loc[:,'year_of_first_registraion']=''
df.loc[:,'value']= df['value']
df.loc[:,'unit']= 'nr'
df.loc[:,'source']= nb_data_source
df.loc[:,'accessed']= '2020-06-30'
df.loc[:,'notebook']= nb_name
df.loc[:,'footnote']= ''

In [17]:
# create a finalized dataframe for output with columns in arranged order
heading_list= ['year_of_measurement','date_of_measurement','geo','process','vehicle_class','vehicle_segment','motor_energy','model_year','year_of_first_registraion','value','unit','source','accessed','notebook','footnote']
df_out= df[heading_list]

In [18]:
#check df_out before saving
df_out

,year_of_measurement,date_of_measurement,geo,process,vehicle_class,vehicle_segment,motor_energy,model_year,year_of_first_registraion,value,unit,source,accessed,notebook,footnote
003000000000,2005,,Austria,r,OICV,all,all,all,,367000,nr,OICA,2020-06-30,003_s_GL_2005_2015_cl(cv)_oica.ipynb,
003000000001,2005,,Belgium,r,OICV,all,all,all,,674465,nr,OICA,2020-06-30,003_s_GL_2005_2015_cl(cv)_oica.ipynb,
003000000002,2005,,Denmark,r,OICV,all,all,all,,479000,nr,OICA,2020-06-30,003_s_GL_2005_2015_cl(cv)_oica.ipynb,
003000000003,2005,,Finland,r,OICV,all,all,all,,86690,nr,OICA,2020-06-30,003_s_GL_2005_2015_cl(cv)_oica.ipynb,
003000000004,2005,,France,r,OICV,all,all,all,,6198000,nr,OICA,2020-06-30,003_s_GL_2005_2015_cl(cv)_oica.ipynb,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
003000001546,2015,,Togo,r,OICV,all,all,all,,58000,nr,OICA,2020-06-30,003_s_GL_2005_2015_cl(cv)_oica.ipynb,
003000001547,2015,,Tunisia,r,OICV,all,all,all,,460000,nr,OICA,2020-06-30,003_s_GL_2005_2015_cl(cv)_oica.ipynb,
003000001548,2015,,Uganda,r,OICV,all,all,all,,340000,nr,OICA,2020-06-30,003_s_GL_2005_2015_cl(cv)_oica.ipynb,
003000001549,2015,,Zambia,r,OICV,all,all,all,,120000,nr,OICA,2020-06-30,003_s_GL_2005_2015_cl(cv)_oica.ipynb,


## at this point restart kernel and run all cells

In [19]:
# write an output file named according to notebook id with relevant data and metadata
writer = pd.ExcelWriter(nb_output_workbook, engine='xlsxwriter')
df_out.to_excel(writer, sheet_name='data', merge_cells=False)
notebook_metadata.to_excel(writer, sheet_name= 'notebook_metadata')
writer.save()

In [20]:
# add the data from the notebook to the stock dataframe and notebook metadata pickles

import pickle
# load in the stock DataFrame pickle
stock_pickle = pd.read_pickle('stock_df.pickle')

# concatenate the out_df to the stock_pickle and remove any duplicate rows
# here it is important to think about the categories of duplicates to drop.
stock_df = pd.concat([df_out,stock_pickle]).drop_duplicates(subset=['notebook','value','year_of_measurement'])

# write the updated stock dataframe to pickle
stock_df.to_pickle('stock_df.pickle')

# repeat the process for the metadata
metadata_pickle = pd.read_pickle('metadata_df.pickle')

metadata_df = pd.concat([metadata_pickle, notebook_metadata], sort=False).drop_duplicates()

metadata_df.to_pickle('metadata_df.pickle')

,year_of_measurement,date_of_measurement,geo,process,vehicle_class,vehicle_segment,motor_energy,model_year,year_of_first_registraion,value,unit,source,accessed,notebook,footnote


In [22]:
# update the stock metadata 
writer = pd.ExcelWriter('vehicle_fleet_stock.xlsx', engine='xlsxwriter')
stock_df.to_excel(writer, sheet_name='data', merge_cells=False)
metadata_df.to_excel(writer, sheet_name= 'notebook_metadata')
writer.save()